## Tacotron 2 inference code 
Edit the variables **checkpoint_path** and **text** to match yours and run the entire code to generate plots of mel outputs, alignments and audio synthesis from the generated mel-spectrogram using Griffin-Lim.

#### Import libraries and setup matplotlib

In [1]:
import matplotlib
%matplotlib inline
import matplotlib.pylab as plt

import IPython.display as ipd

import sys
sys.path.append('waveglow/')
import numpy as np
import torch

from hparams import create_hparams
from model import Tacotron2
from layers import TacotronSTFT, STFT
from audio_processing import griffin_lim
from train import load_model
from text import text_to_sequence
from denoiser import Denoiser

In [2]:
torch.cuda.get_device_capability(0)

(8, 9)

In [3]:
def plot_data(data, figsize=(16, 4)):
    fig, axes = plt.subplots(1, len(data), figsize=figsize)
    for i in range(len(data)):
        axes[i].imshow(data[i], aspect='auto', origin='bottom',
                       interpolation='none')

#### Setup hparams

In [4]:
hparams = create_hparams()
hparams.sampling_rate = 22050

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



#### Load model from checkpoint

In [5]:
checkpoint_path = "outdir/checkpoint_121000"
model = load_model(hparams)
model.load_state_dict(torch.load(checkpoint_path)['state_dict'])
model.cuda().eval()

Tacotron2(
  (embedding): Embedding(37, 512)
  (encoder): Encoder(
    (convolutions): ModuleList(
      (0): Sequential(
        (0): ConvNorm(
          (conv): Conv1d(512, 512, kernel_size=(5,), stride=(1,), padding=(2,))
        )
        (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): Sequential(
        (0): ConvNorm(
          (conv): Conv1d(512, 512, kernel_size=(5,), stride=(1,), padding=(2,))
        )
        (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (2): Sequential(
        (0): ConvNorm(
          (conv): Conv1d(512, 512, kernel_size=(5,), stride=(1,), padding=(2,))
        )
        (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (lstm): LSTM(512, 256, batch_first=True, bidirectional=True)
  )
  (decoder): Decoder(
    (prenet): Prenet(
      (layers): ModuleList(
        (0): LinearNorm(
          (line

#### Load WaveGlow for mel2audio synthesis and denoiser

In [6]:
# waveglow_path = 'waveglow/checkpoints/waveglow_140000'
# waveglow = torch.load(waveglow_path)['model']
# waveglow.cuda().eval()
# for k in waveglow.convinv:
#     k.float()
# denoiser = Denoiser(waveglow)

#### Prepare text input

In [7]:
import os
import shutil

line = "data/minji-kim_bwv1003_mov2_95.wav|90 D4e,H4e,A4e,A5e A4s C4s A4e A5e ] D4e,H4e,A4e,H5e D5e C3e,L4e,F5e C4e ] A3e,A4e,D5e I4e K4e,E5e H4e"
score, text = line.split('|')

text = text.replace('\n', '')
print(text)
shutil.copy2(score, f'waveglow/mels/{score}_ground.wav')

sequence = np.array(text_to_sequence(text, ['ptp_cleaners']))[None, :]
sequence = torch.autograd.Variable(
    torch.from_numpy(sequence)).cuda().long()

mel_outputs, mel_outputs_postnet, _, alignments = model.inference(sequence)

a = mel_outputs.float().data.cpu()[0]
b = mel_outputs_postnet.float().data.cpu()[0]
c = alignments.float().data.cpu()[0].T

torch.save(b, f'waveglow/mels/{score}_postnet.pt')
with open(f'waveglow/mels/{score}.txt', 'w') as f:
    f.write(text)

plot_data((mel_outputs.float().data.cpu().numpy()[0],
        mel_outputs_postnet.float().data.cpu().numpy()[0],
        alignments.float().data.cpu().numpy()[0].T))

90 D4e,H4e,A4e,A5e A4s C4s A4e A5e ] D4e,H4e,A4e,H5e D5e C3e,L4e,F5e C4e ] A3e,A4e,D5e I4e K4e,E5e H4e


#### Decode text input and plot results

#### Synthesize audio from spectrogram using WaveGlow

In [8]:
# with torch.no_grad():
#     audio = waveglow.infer(mel_outputs_postnet, sigma=0.666)
# ipd.Audio(audio[0].data.cpu().numpy(), rate=hparams.sampling_rate)

#### (Optional) Remove WaveGlow bias

In [9]:
# audio_denoised = denoiser(audio, strength=0.01)[:, 0]
# ipd.Audio(audio_denoised.cpu().numpy(), rate=hparams.sampling_rate)